In [5]:
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org ydata-profiling
#!pip install missingno --trusted-host pypi.org --trusted-host files.pythonhosted.org

import pandas as pd
import warnings
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from ydata_profiling import ProfileReport
import missingno as msno


In [7]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

archivo_2020_1 = 'data/Saber_11__2020-1_20240828.csv'
archivo_2020_2 = 'data/Saber_11__2020-2_20240828.csv'

saber_2020_1 = pd.read_csv(archivo_2020_1)
saber_2020_2 = pd.read_csv(archivo_2020_2)


saber_2020 = pd.concat([saber_2020_1, saber_2020_2], ignore_index=True)

saber_2020.head()

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,PERIODO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_PAIS_RESIDE,ESTU_TIENEETNIA,ESTU_DEPTO_RESIDE,ESTU_COD_RESIDE_DEPTO,ESTU_MCPIO_RESIDE,ESTU_COD_RESIDE_MCPIO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TRABAJOLABORPADRE,FAMI_TRABAJOLABORMADRE,FAMI_TIENEINTERNET,FAMI_TIENESERVICIOTV,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_NUMLIBROS,FAMI_COMELECHEDERIVADOS,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_SITUACIONECONOMICA,ESTU_DEDICACIONLECTURADIARIA,ESTU_DEDICACIONINTERNET,ESTU_HORASSEMANATRABAJA,ESTU_TIPOREMUNERACION,COLE_CODIGO_ICFES,COLE_COD_DANE_ESTABLECIMIENTO,COLE_NOMBRE_ESTABLECIMIENTO,COLE_GENERO,COLE_NATURALEZA,COLE_CALENDARIO,COLE_BILINGUE,COLE_CARACTER,COLE_COD_DANE_SEDE,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,COLE_AREA_UBICACION,COLE_JORNADA,COLE_COD_MCPIO_UBICACION,COLE_MCPIO_UBICACION,COLE_COD_DEPTO_UBICACION,COLE_DEPTO_UBICACION,ESTU_PRIVADO_LIBERTAD,ESTU_COD_MCPIO_PRESENTACION,ESTU_MCPIO_PRESENTACION,ESTU_DEPTO_PRESENTACION,ESTU_COD_DEPTO_PRESENTACION,PUNT_LECTURA_CRITICA,PERCENTIL_LECTURA_CRITICA,DESEMP_LECTURA_CRITICA,PUNT_MATEMATICAS,PERCENTIL_MATEMATICAS,DESEMP_MATEMATICAS,PUNT_C_NATURALES,PERCENTIL_C_NATURALES,DESEMP_C_NATURALES,PUNT_SOCIALES_CIUDADANAS,PERCENTIL_SOCIALES_CIUDADANAS,DESEMP_SOCIALES_CIUDADANAS,PUNT_INGLES,PERCENTIL_INGLES,DESEMP_INGLES,PUNT_GLOBAL,PERCENTIL_GLOBAL,ESTU_ESTADOINVESTIGACION,ESTU_GENERACION-E,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO
0,CC,COLOMBIA,F,01/01/1985,20201,SB11202010045555,ESTUDIANTE,COLOMBIA,No,CESAR,20.0,SAN DIEGO,20750.0,Estrato 1,5 a 6,Uno,Primaria incompleta,Primaria incompleta,"Trabaja en el hogar, no trabaja o estudia",Trabaja por cuenta propia (por ejemplo plomero...,No,No,No,No,No,No,No,No,0 A 10 LIBROS,1 o 2 veces por semana,1 o 2 veces por semana,Todos o casi todos los días,Mejor,No leo por entretenimiento,No Navega Internet,Más de 30 horas,"Si, en efectivo",57372,120750000415,I.E. MANUEL RODRIGUEZ TORICES,MIXTO,OFICIAL,A,S,TÉCNICO/ACADÉMICO,120750000415,I.E. MANUEL RODRIGUEZ TORICES,S,URBANO,NOCHE,20750,SAN DIEGO,20,CESAR,N,20001.0,VALLEDUPAR,CESAR,20.0,39,6,2,32,4,1,35,6,1,24,1,1,36.0,7,A-,164,2.0,VALIDEZ OFICINA JURÍDICA,GENERACION E - GRATUIDAD,NaN,NaN,NaN
1,CC,COLOMBIA,F,01/01/1995,20201,SB11202010045719,ESTUDIANTE,COLOMBIA,No,NARIÑO,52.0,IPIALES,52356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177618,352356001951,INSTITUTO DE EDUCACIÓN TECNICA INESUR,MIXTO,NO OFICIAL,A,NaN,NaN,352356001951,INSTITUTO DE EDUCACIÓN TECNICA INESUR - SEDE P...,S,URBANO,SABATINA,52356,IPIALES,52,NARIÑO,N,52356.0,IPIALES,NARIÑO,52.0,41,7,2,41,12,2,39,10,1,44,23,2,30.0,3,A-,202,10.0,PUBLICAR,GENERACION E - GRATUIDAD,NaN,NaN,NaN
2,CC,COLOMBIA,F,01/01/1997,20201,SB11202010070662,ESTUDIANTE,COLOMBIA,Si,CAUCA,19.0,TOTORÓ,19824.0,Estrato 1,3 a 4,Cuatro,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) incompleta,"Trabaja en el hogar, no trabaja o estudia","Trabaja en el hogar, no trabaja o estudia",No,No,No,No,No,No,No,No,0 A 10 LIBROS,1 o 2 veces por semana,NaN,NaN,Igual,30 minutos o menos,No Navega Internet,Entre 11 y 20 horas,"Si, en efectivo",135301,319001004669,COORPORACION EDUCATIVA DEL SUR OCCIDENTE COLOM...,MIXTO,NO OFICIAL,OTRO,NaN,ACADÉMICO,319001004669,COORPORACION EDUCATIVA DEL SUR OCCIDENTE COLOM...,S,URBANO,MAÑANA,19001,POPAYÁN,19,CAUCA,N,19001.0,POPAYÁN,CAUCA,19.0,37,4,2,36,7,2,33,4,1,24,1,1,30.0,3,A-,162,2.0,PUBLICAR,GENERACION E - GRATUIDAD,NaN,NaN,NaN
3,CC,COLOMBIA,F,01/01/2001,20201,SB11202010069926,ESTUDIANTE,COLOMBIA,No,PUTUMAYO,86.0,MOCOA,86001.0,Estrato 1,1 a 2,Uno,Técnica o tecnológica completa,Educación profesional completa,"Es dueño de un negocio grande, tiene un cargo ...","Trabaja como profesional (por ejemplo médico, ...",Si,No,

In [10]:
missing_summary = pd.DataFrame({
    'Missing Count': saber_2020.isnull().sum(),
    'Missing Percentage': (saber_2020.isnull().sum() / len(saber_2020)) * 100
})

# Filter to show only columns with missing values
missing_summary = missing_summary[missing_summary['Missing Count'] > 0]

# Sort by the number of missing values
missing_summary = missing_summary.sort_values(by='Missing Count', ascending=False)

# Display the summary
display(missing_summary)

,Missing Count,Missing Percentage
COLE_BILINGUE,85622,16.456054
ESTU_NSE_INDIVIDUAL,29380,5.646666
ESTU_INSE_INDIVIDUAL,29380,5.646666
FAMI_COMECEREALFRUTOSLEGUMBRE,21811,4.191948
FAMI_TIENECOMPUTADOR,21616,4.154470
FAMI_TRABAJOLABORPADRE,21147,4.064331
FAMI_TIENEAUTOMOVIL,19264,3.702430
ESTU_TIPOREMUNERACION,19104,3.671678
FAMI_TRABAJOLABORMADRE,19059,3.663030
FAMI_TIENECONSOLAVIDEOJUEGOS,18929,3.638044


In [8]:
profile = ProfileReport(saber_2020, title="Exploración Datos Pruebas Saber", explorative=True)

# Save the report to an HTML file
profile.to_file("estadisticasDescriptivasSaber2020.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]